# Prompt and Response Construction Experiments

## Imports and setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import sys
from pathlib import Path

# Add backend folder to Python path
backend_path = Path.cwd().parent  # Goes from nbs/ to backend/
sys.path.insert(0, str(backend_path))

In [4]:
from importlib import reload
import app.submodules
import importlib
import app.utils
import app.rag_utils
import app.submodules

/Users/oliviacheng/miniconda3/envs/cspnj/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/oliviacheng/miniconda3/envs/cspnj/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [5]:
from app.submodules import construct_response

## Basic Example

In [6]:
TEXT = "I am located in Vineland, NJ off of Landis Avenue. Do you have information on the soup kitchens in Vineland? I want specific links and addresses"
PREVIOUS_TEXT = []
MODEL = "copilot"
ORG = "cspnj"

In [7]:
generator = construct_response(TEXT, PREVIOUS_TEXT, MODEL, ORG)

In [8]:
full_response = ""
for chunk in generator:
    full_response += chunk.replace("\n\n","").replace("data: ","")

[DEBUG] needs_goals=True, verbosity=deep
[DEBUG] verbosity=deep → using full orchestration with k=50
[DEBUG] copilot pipeline branch (SMART goals + orchestration)


/Users/oliviacheng/projects/research/community_services_llm/backend/app/rag_utils.py:142: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  resources_df = pd.read_sql_query("""


All messages [{'role': 'system', 'content': '# Wellbeing SMART Goal Assistant Prompt\n\nYou are a thoughtful and knowledgeable chatbot associated with **cspnj**, designed to help clients with their overall wellbeing.  \nYou guide center members along the **eight dimensions of wellness**: *emotional, physical, occupational, social, spiritual, intellectual, environmental, and financial.*\n\n---\n\n## Task\n\nAt this stage, your role is to **ask purposeful questions** that help clarify the member’s situation and guide them toward meaningful **SMART (Specific, Measurable, Achievable, Realistic, and Timely) goals**.\n\nDo **not** provide resources or direct solutions yet — focus only on gathering the right information.\n\n---\n\n## Guidelines\n\n1. **Ask Insightful, Detailed Questions**  \n   - Include questions about the member’s **location**, **current situation**, and any **context** that would help shape their SMART goals.  \n   - For each question, provide a **brief explanation** of wh

In [9]:
formatted_response = full_response.replace("<br/>","\n")

In [10]:
print(formatted_response)

It sounds like you're doing a wonderful job of thinking through how to assist in your community while also supporting your own wellbeing! There are several soup kitchens and food assistance resources in and around Vineland, NJ, that align well with your goals of volunteering and preparing meals for donation.

Here are some options you can explore:

1. **Salvation Army Vineland**  
   - **Address:** 733 E. Landis Avenue, Vineland, NJ 08360  
   - **Phone:** 856-696-5050  
   - **Website:** [Salvation Army Vineland](https://www.salvationarmy.org)  

2. **Vineland Elks Lodge #1422**  
   - **Address:** 1855 E. Landis Avenue, Vineland, NJ 08361  
   - **Phone:** 856-691-1641  
   - **Website:** [Vineland Elks Lodge](http://www.elks.org/)  

3. **Trinity First Hope Center** (located in nearby Millville)  
   - **Address:** 100 S 2nd St, Millville, NJ 08332  
   - **Phone:** 856-825-2329  
   - **Website:** [Trinity First Hope Center](https://www.firstumcmillville.org/trinity-first-hope-cent

In [11]:
import pandas as pd
import os
import psycopg

In [12]:
CONNECTION_STRING = os.getenv('RESOURCE_DB_URL')
with psycopg.connect(CONNECTION_STRING) as conn:
    df = pd.read_sql_query("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public'
    """, conn)
    print(df)

/var/folders/lf/lgn5wxd97mzcmfjqdkjmq1dh0000gn/T/ipykernel_92456/602818015.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("""


  table_name
0  resources


In [13]:
with psycopg.connect(CONNECTION_STRING) as conn:
    df = pd.read_sql_query("""
        SELECT * from resources
    """, conn)

/var/folders/lf/lgn5wxd97mzcmfjqdkjmq1dh0000gn/T/ipykernel_92456/2679618489.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("""


In [14]:
df.head()

,id,organization,service,description,url,phone,embedding
0,1,cspnj,262-HELP (Crisis Response Service),NaN,NaN,NaN,"[-0.0015057984,-0.04301608,0.015712785,-0.0125..."
1,2,cspnj,(NA) Narcotics Anonymous,Narcotics Anonymous of New Jersey provides sup...,https://www.nanj.org/,8009920401,"[-0.034808174,0.07251281,0.008178203,-0.015189..."
2,3,cspnj,(211) Food Pantries/Soup Kitchens,The NJ 2-1-1 Partnership provides information ...,https://nj211.org/food-pantries-and-soup-kitchens,NaN,"[-0.008280203,0.06520874,-0.0067741363,-0.0098..."
3,4,cspnj,(NA) Recovery,Narcotics Anonymous of New Jersey offers suppo...,https://www.nanj.org/,8009920401,"[-0.027628645,0.08028611,-0.009514642,-0.02050..."
4,5,cspnj,211 Code Blue Alert - NJ Warming Centers,New Jersey Code Blue is a program activated du...,https://nj211.org/njcodeblue,8776521148,"[-0.027639408,-0.0037133705,0.00823962,0.02425..."


In [23]:
for i, s in enumerate(df['description']):
    if "elks lodge" in s.lower():
        print(df.iloc[i])

In [22]:
for i, s in enumerate(df['description']):
    if "salvation army vineland" in s.lower():
        print(df.iloc[i])

id                                                           1000
organization                                                cspnj
service              Salvation Army of New Jersey- Vineland Corps
description     The Salvation Army Vineland, located at 733 Ea...
url             https://easternusa.salvationarmy.org/new-jerse...
phone                                                  8566965050
embedding       [0.0011713024,0.011972207,0.032044627,0.024858...
Name: 1000, dtype: object


In [24]:
for i, s in enumerate(df['description']):
    if "trinity first hope" in s.lower():
        print(df.iloc[i])

id                                                           1186
organization                                                cspnj
service                                 Trinity First Hope Center
description     Trinity First Hope Center is located at 100 S ...
url             https://www.firstumcmillville.org/trinity-firs...
phone                                                  8568252329
embedding       [-0.037914954,0.036107354,-0.0012465805,0.0155...
Name: 1187, dtype: object


## Summary of test:

Bot provides correct *resource name* and *URL* and *phone*.

Note that even though the resource database contained an incorrect link for Salvation Army Vineland, the AI did not use that link. The AI just provided the general Salvation Army link. 